# Encoding

In [10]:
import os
import re
import sys
import warnings

import copy
import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder

warnings.filterwarnings('ignore')

# define functions

In [11]:
def ordinal_encode(df, df_test, testing = False):
#     df = df.copy()
    categories_dict = {}
    
    temp_merge = df.append(df_test)
    for cat in temp_merge.columns:
        if temp_merge[cat].dtypes == 'object':
            categories_dict[cat] = list(temp_merge[cat].unique())
            if testing:
                print("Numero de categorias para variavel '{}': {} ".format(cat,temp_merge[cat].unique().size))

    if testing:
        print()
        print(list(categories_dict.keys()))
        
    enc = OrdinalEncoder(categories=list(categories_dict.values()))
    trained_encoder = enc.fit(df[list(categories_dict.keys())])
    
    # transform train and test
    df[list(categories_dict.keys())] = trained_encoder.transform(df[list(categories_dict.keys())])
    df_test[list(categories_dict.keys())] = trained_encoder.transform(df_test[list(categories_dict.keys())])

    if testing:
        print(categories_dict)
    
    return df, df_test

def one_hot_encode(df):
    print('Quantity of columns before one-hot encoding:', len(df.columns))
    
    df_oldcols = df.columns.to_list()
    df = pd.get_dummies(df, prefix_sep='_', drop_first=True)
    
    print('Quantity of columns after one-hot encoding:', len(df.columns))
    
    # rename columns to show which are dummies
    onehot_cols = list(set(df.columns.to_list()) - set(df_oldcols))
    onehot_cols_renaming = {col: 'dummy_'+col.replace('-', '_') for col in onehot_cols}
    df.rename(columns = onehot_cols_renaming, inplace=True)
    
    return df

# Define paths and capture data

In [12]:
inputs = os.path.join('..', 'data', '02_intermediate')
outputs = os.path.join('..', 'data', '03_processed')
reports = os.path.join('..', 'data', '06_reporting')

ord_dict = {}
ord_dict['X_train'] = pd.read_csv(os.path.join(inputs, 'X_train.csv'), index_col='id')
ord_dict['X_test'] = pd.read_csv(os.path.join(inputs, 'X_test.csv'), index_col='id')

onehot_dict = copy.deepcopy(ord_dict)

y_train = pd.read_csv(os.path.join(inputs, 'y_train.csv'), index_col='id') 
y_test = pd.read_csv(os.path.join(inputs, 'y_test.csv'), index_col='id')

# Count categorical data unique values
Check both train and test. Any inconsistency between them should be addressed.

In [13]:
for data in ['X_train', 'X_test']:
    categories_dict = {}
    print('\r\nchecking number of categories for {}'. format(data))
    for cat in ord_dict[data].columns:
        if ord_dict[data][cat].dtypes == 'object':
            categories_dict[cat] = list(ord_dict[data][cat].unique())
            print("Numero de categorias para variavel '{}': {} ".format(cat, ord_dict[data][cat].unique().size))


checking number of categories for X_train
Numero de categorias para variavel 'productcd': 5 
Numero de categorias para variavel 'card4': 4 
Numero de categorias para variavel 'card6': 2 
Numero de categorias para variavel 'p_emaildomain': 54 
Numero de categorias para variavel 'r_emaildomain': 40 
Numero de categorias para variavel 'm4': 3 

checking number of categories for X_test
Numero de categorias para variavel 'card4': 4 
Numero de categorias para variavel 'card6': 3 
Numero de categorias para variavel 'm4': 3 
Numero de categorias para variavel 'p_emaildomain': 45 
Numero de categorias para variavel 'productcd': 5 
Numero de categorias para variavel 'r_emaildomain': 29 


# Ordinal Encoding

In [14]:
ord_dict['X_train'], ord_dict['X_test'] = ordinal_encode(ord_dict['X_train'], ord_dict['X_test'], testing = False)

ord_dict['X_train'].head()

,transactiondt,transactionamt,productcd,card1,card2,card3,card4,card5,card6,addr1,...,m5,m6,m7,m8,m9,if_anomaly,transactionamt_to_mean_card1,transactionamt_to_mean_card4,transactionamt_to_std_card1,transactionamt_to_std_card4
id,,,,,,,,,,,,,,,,,,,,,
3486774,13107389.0,38.056,0.0,9633.0,130.0,185.0,0.0,138.0,0.0,270.239972,...,0.520700,0.497537,0.124312,0.507226,0.965394,1,0.845360,0.280580,0.710967,0.160854
3062695,1650884.0,150.000,1.0,15063.0,514.0,150.0,0.0,226.0,1.0,194.000000,...,0.491038,0.463338,0.110642,0.328999,0.869981,1,0.598610,1.105924,0.356718,0.634016
3273443,7048761.0,56.500,2.0,9006.0,555.0,143.0,1.0,224.0,0.0,502.000000,...,0.000000,0.342737,0.000000,0.000000,1.000000,1,1.000000,0.446743,NaN,0.234439
3384445,10011292.0,8.459,0.0,11201.0,103.0,185.0,0.0,226.0,0.0,300.433901,...,0.533395,0.454835,0.116979,0.444016,0.857350,1,0.190744,0.062367,0.289492,0.035754
3489059,13159069.0,77.950,2.0,7919.0,194.0,150.0,1.0,166.0,0.0,315.000000,...,0.345176,1.000000,0.000000,0.000000,0.000000,1,0.854398,0.616348,0.775027,0.323443


# One-Hot Encoding

In [15]:
for df in ['X_train', 'X_test']:
    onehot_dict[df] = one_hot_encode(onehot_dict[df])
    
print('\r\nColumns of the new database:')
# print(onehot_dict[df].columns.to_list())

Quantity of columns before one-hot encoding: 50
Quantity of columns after one-hot encoding: 146
Quantity of columns before one-hot encoding: 58
Quantity of columns after one-hot encoding: 135

Columns of the new database:


# report new data types

### data alignment
if some category is missing on test set, we need to account for that and build corresponding column filled with 'zeros'.

In [16]:
def fill_missing_cols(smaller, greater):
    missing_cols = set( greater.columns ) - set( smaller.columns )
    for c in missing_cols:
        smaller[c] = 0
    
    return smaller

In [17]:
onehot_dict['X_train'] = fill_missing_cols(onehot_dict['X_train'], onehot_dict['X_test'])
onehot_dict['X_test'] = fill_missing_cols(onehot_dict['X_test'], onehot_dict['X_train'])

# align column positions (no data leakage here. Just altering column ordering.)
onehot_dict['X_train'], onehot_dict['X_test'] = onehot_dict['X_train'].align(onehot_dict['X_test'], axis=1)

# Save processed data

In [18]:
for df in ['X_train', 'X_test']:
    ord_dict[df].to_csv(os.path.join(outputs, df+'.csv'))
    onehot_dict[df].to_csv(os.path.join(outputs, df+'_onehot.csv'))
    
for df in ['X_train', 'X_test']:
    print(ord_dict[df].shape)
    print(onehot_dict[df].shape)
    
y_train.to_csv(os.path.join(outputs, 'y_train.csv'))
y_test.to_csv(os.path.join(outputs, 'y_test.csv'))

(7000, 50)
(7000, 162)
(3000, 58)
(3000, 162)


# save report over data types